In [36]:
import pandas as pd
import numpy as np
import sys
import os
import cv2

import matplotlib.pyplot as plt

from skimage import io,filters

In [2]:
IMAGES_DIR = "./datasets/Plate_Image"
GRAYED_DIR = "./outputs/Grayed_Image"
LABELS_FILE = "./datasets/Plate_Index.xls"

In [4]:
xls_file = pd.ExcelFile(LABELS_FILE)
xls_file.sheet_names

['Sheet1', 'Sheet2', 'Sheet3']

In [5]:
data = xls_file.parse("Sheet1")
data.columns = ['Image ID', 'Corner1_x', 'Corner1_y', 'Corner2_x', 'Corner2_y', 'Corner3_x', 'Corner3_y', 
                'Corner4_x', 'Corner4_y', 'Char1', 'Char2', 'Char3','Char4', 'Char5', 'Char6', 'Char7']

In [6]:
data.head()

,Image ID,Corner1_x,Corner1_y,Corner2_x,Corner2_y,Corner3_x,Corner3_y,Corner4_x,Corner4_y,Char1,Char2,Char3,Char4,Char5,Char6,Char7
0,702172519671,336,323,688,310,682,412,339,430,10,15,12,8,5,5,0
1,702172544593,301,286,728,291,725,423,306,418,10,23,26,1,9,6,9
2,702172623671,294,299,684,322,675,411,299,392,10,20,21,8,3,9,9
3,702172729578,200,267,723,291,707,398,222,381,10,21,22,7,9,2,5
4,702172759468,348,223,608,278,599,359,346,305,10,21,29,2,0,1,1


In [8]:
fnames = [os.path.join(IMAGES_DIR, str(fname) + ".jpg") for fname in data["Image ID"]]

In [54]:
for i in range(5):
    plate_x_l = min(data["Corner1_x"][i], data["Corner4_x"][i])
    plate_x_r = max(data["Corner2_x"][i], data["Corner3_x"][i])
    plate_y_u = min(data["Corner1_y"][i], data["Corner2_y"][i])
    plate_y_d = max(data["Corner3_y"][i], data["Corner4_y"][i])
    path = fnames[i]
    img = io.imread(path, as_gray=True)
    cropImg = img[plate_y_u-15:plate_y_d+15, plate_x_l-30:plate_x_r+30]
    thresh = filters.threshold_otsu(cropImg)
    dst = (cropImg > thresh) * 1
    if not os.path.exists(GRAYED_DIR):
        os.mkdir(GRAYED_DIR)     
    path_out = os.path.join(GRAYED_DIR, str(i+1)+".png")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)